In [46]:
#import modules
import numpy as np
import pandas as pd
import folium 
import requests
from pandas.io.json import json_normalize
from pandas import DataFrame
from folium import plugins

In [47]:
#sample world map
world_map = folium.Map()
world_map

In [48]:
#call api
url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"

querystring = {"sort":"sold_date","city":"Alliance","offset":"0","state_code":"OH","limit":"200"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "1e794bfadcmshb631310927ae8d8p12e6b2jsn9adc4a4ac445"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


data = response.json()


In [49]:
#normalize data and filter by lat,long and price.
normal = pd.json_normalize(data, ['properties'])
prototype = DataFrame(normal, columns = ['address.lat','address.lon','price'])
#drop all the NaN values
df = prototype.dropna()
df

,address.lat,address.lon,price
0,40.939969,-81.123584,30000
1,40.904595,-81.123573,148000
2,40.922272,-81.119794,130000
3,40.893851,-81.137430,96500
4,40.898339,-81.122680,75000
...,...,...,...
195,40.906804,-81.131079,198500
196,40.892980,-81.031618,167000
197,40.899510,-81.103214,125000
198,40.933789,-81.103404,95000


In [50]:
#Create the map 
money = folium.map.FeatureGroup()
for lat,long, in zip(df['address.lat'],df['address.lon']):
    money.add_child(
    folium.CircleMarker(
    [lat,long],
    radius=5,
    color='yellow',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6
    
        )
    
    )


In [51]:
world_map.add_child(money)

In [52]:
#make variables
latitudes = list(df['address.lat'])
longitudes = list(df['address.lon'])
labels = list(df['price'])
#make heatmap cluster
cluster = plugins.MarkerCluster().add_to(world_map)
for lat, long, label in zip(latitudes,longitudes,labels):
    folium.Marker([lat, long], popup=label).add_to(cluster)



In [53]:
world_map